In [0]:
# configs = {"fs.azure.account.key.storage001de.blob.core.windows.net": "MA38y0ZA2Ij1L3QXl/F/Y8EGXR1VjTGb896PNbUmPT99uhVLoi0qCEiBEBagBQ4yscqwq9Zxb0vX+AStlr0Z3Q=="}

# # create mount to silver
# dbutils.fs.mount(
# 	source = "wasbs://silver@storage001de.blob.core.windows.net/",
# 	mount_point = "/mnt/datalake/silver",
# 	extra_configs = configs
# )

In [0]:
df_director = spark.read.table('bronze.director_to_movie')
df_actor = spark.read.table('bronze.actor')

In [0]:
df_director.show(5)
df_actor.show(5)

+-------+-------+
|actorId|movieId|
+-------+-------+
|     47|  58555|
|     47|  60997|
|     47|  66113|
|     52|    229|
|     52|    275|
+-------+-------+
only showing top 5 rows

+---+--------------------+--------------------+----------+---------+----------------+--------------------+--------+----------+----------+------+
| id|          created_at|          updated_at|deleted_at|   imdbId|            name|            imageUrl|birthDay|birthPlace|partialBio|height|
+---+--------------------+--------------------+----------+---------+----------------+--------------------+--------+----------+----------+------+
|  1|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|nm0005169|  Natasha Lyonne|https://m.media-a...|    NULL|      NULL|      NULL|  NULL|
|  2|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|nm0925966|Bradley Whitford|https://m.media-a...|    NULL|      NULL|      NULL|  NULL|
|  3|2023-11-01 11:54:...|2023-11-01 11:54:...|      NULL|nm0925717|Paxton Whitehead|htt

In [0]:
df_director = df_director.dropDuplicates()
df_actor = df_actor.dropDuplicates(['id'])

### Rename

In [0]:
df_director = df_director.withColumnRenamed('actorId', 'id')
df_actor = df_actor.withColumnRenamed('name', 'director')

#### Join

In [0]:
df_new = df_director.join(df_actor , on=['id'] , how = 'left')
df_new.show()

+---+-------+--------------------+--------------------+----------+---------+--------------------+--------------------+--------+----------+----------+------+
| id|movieId|          created_at|          updated_at|deleted_at|   imdbId|            director|            imageUrl|birthDay|birthPlace|partialBio|height|
+---+-------+--------------------+--------------------+----------+---------+--------------------+--------------------+--------+----------+----------+------+
| 47|  58555|2023-11-01 11:55:...|2023-11-01 11:55:...|      NULL|nm0359894|         Roger Hanin|https://m.media-a...|    NULL|      NULL|      NULL|  NULL|
| 47|  58555|2023-11-01 11:55:...|2023-11-01 11:55:...|      NULL|nm0359894|         Roger Hanin|https://m.media-a...|    NULL|      NULL|      NULL|  NULL|
| 47|  60997|2023-11-01 11:55:...|2023-11-01 11:55:...|      NULL|nm0359894|         Roger Hanin|https://m.media-a...|    NULL|      NULL|      NULL|  NULL|
| 47|  60997|2023-11-01 11:55:...|2023-11-01 11:55:...|   

In [0]:
df_new = df_new.select('movieId', 'director')

#### Get first director

In [0]:
df_new = df_new.dropDuplicates(subset=['movieId', 'director'])
df_new = df_new.sort('movieId')

df_new.show()
df_new.count()

+-------+--------------------+
|movieId|            director|
+-------+--------------------+
|      1|       James Mangold|
|      2|         John Lemont|
|      3|      Georges Franju|
|      4|     Jean-Luc Godard|
|      5|      William Castle|
|      6|          John Wayne|
|      7|        Billy Wilder|
|      8|Michelangelo Anto...|
|      9|         Daniel Mann|
|     10|         Walter Lang|
|     11|        Anthony Mann|
|     11|     Charles Walters|
|     12|    Federico Fellini|
|     13|      Richard Brooks|
|     14|      Otto Preminger|
|     15|C.M. Pennington-R...|
|     16|      Ingmar Bergman|
|     17|        John Sturges|
|     18|          Mario Bava|
|     19|        David Miller|
+-------+--------------------+
only showing top 20 rows



115601

#### Save table

In [0]:
# Define the silver location
director_silverLocation = "/mnt/datalake/silver/cleaned_director"

# Table name
director_bronzeTable = "bronze.director"
director_silverTable = "silver.director"

In [0]:
from delta.tables import *
from pyspark.sql.functions import col, substring

# #check if the silver location contain the delta table
# if (DeltaTable.isDeltaTable(spark, movie_silverLocation)):
#     DeltaTable.forPath(spark, movie_silverLocation).alias("target").merge(
#         source = cleaned_movie.alias("src"),
#         condition = "target.id = src.id"
#     ).WhenNoMatchedInsertAll().execute()
# else:

df_new.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save(director_silverLocation)
    

In [0]:
spark.sql(f"CREATE EXTERNAL TABLE IF NOT EXISTS {director_silverTable} USING delta LOCATION '/mnt/datalake/silver/cleaned_director'")

DataFrame[]

In [0]:
# Maintenance for Delta table

# To optimized the performance of the Delta table, we need to excute 2 commands:
# 1. optimize(): Optimize the number of files used to store the data
# 2. vacuum(): Remove the old version of the data. This reduces the overhead but it limites the version we can go back to.
director_DataDelta = DeltaTable.forName(spark, director_silverTable)

if director_DataDelta.history(30).filter("operation = 'VACUUM START'").count() == 0:
	director_DataDelta.optimize()
	director_DataDelta.vacuum()


In [0]:
%sql
DESCRIBE HISTORY silver.director;

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
3,2023-12-14T10:50:41Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(2285270932896338),1122-230829-ksqo7xkx,2,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 115601, numOutputBytes -> 1496003)",null,Databricks-Runtime/13.3.x-scala2.12
2,2023-12-13T11:23:35Z,1311347395691033,totnvt02@gmail.com,VACUUM END,Map(status -> COMPLETED),null,List(2285270932896338),1122-230829-ksqo7xkx,1,SnapshotIsolation,true,"Map(numDeletedFiles -> 0, numVacuumedDirectories -> 1)",null,Databricks-Runtime/13.3.x-scala2.12
1,2023-12-13T11:23:31Z,1311347395691033,totnvt02@gmail.com,VACUUM START,"Map(retentionCheckEnabled -> true, defaultRetentionMillis -> 604800000)",null,List(2285270932896338),1122-230829-ksqo7xkx,0,SnapshotIsolation,true,"Map(numFilesToDelete -> 0, sizeOfDataToDelete -> 0)",null,Databricks-Runtime/13.3.x-scala2.12
0,2023-12-13T11:23:16Z,1311347395691033,totnvt02@gmail.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(2285270932896338),1122-230829-ksqo7xkx,null,WriteSerializable,false,"Map(numFiles -> 2, numOutputRows -> 104678, numOutputBytes -> 1345242)",null,Databricks-Runtime/13.3.x-scala2.12


In [0]:

%sql
select * from silver.director

movieId,director
1,James Mangold
2,John Lemont
3,Georges Franju
4,Jean-Luc Godard
5,William Castle
6,John Wayne
7,Billy Wilder
8,Michelangelo Antonioni
9,Daniel Mann
10,Walter Lang
